In [1]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example: 'spark-3.2.2'
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http

In [2]:
# Install pytest and pytest-sugar to make our output look nice.
!pip install -q pytest pytest-sugar

In [3]:
# Create and navigate to the tdd directory.
from pathlib import Path
if Path.cwd().name != 'tests':
    %mkdir tests
    %cd tests
# Show the current working directory.  
%pwd

/content/tests


'/content/tests'

In [4]:
# Initialize the __init__.py file. 
# This file will be stored in our pwd (/content/tests)
%%file __init__.py
pass

Writing __init__.py


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession

import time

spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
energy = spark.read.csv('../drive/MyDrive/organised_Gen.csv', header = True, inferSchema = True)
energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
energy.write.parquet('parquet_energy',mode='overwrite')
p_energy = spark.read.parquet('parquet_energy')
p_energy.createOrReplaceTempView('p_energy_data')

start_time = time.time()
transformed_df = spark.sql("""
    SELECT YEAR as year, producer, source, generated AS energy FROM p_energy_data
    WHERE state != 'US-TOTAL' AND producer != 'Total Electric Power Industry' AND source != 'Total'
        """)
transformed_df

  
q_df = spark.sql("""
      SELECT year, producer, SUM(generated) AS total_generated FROM p_energy_data
      WHERE year >= 2011 AND year < 2022 AND (source='Wind' OR source='Solar Thermal and Photovoltaic' OR source='Hydroelectric Conventional' OR source='Geothermal' OR source='Wood and Wood Derived Fuels' OR source='Other Biomass' OR source='Pumped Storage')
      GROUP BY year, producer
      ORDER BY year DESC, producer DESC;
                """)
q_df


print("--- %s seconds ---" % (time.time() - start_time))

--- 0.055159568786621094 seconds ---


In [12]:
q_df.show()

+----+--------------------+---------------+
|year|            producer|total_generated|
+----+--------------------+---------------+
|2021|Total Electric Po...|  1.642551351E9|
|2021|Electric Generato...|   9.26441207E8|
|2021|Electric Generato...|   6.47537491E8|
|2021|Combined Heat and...|    5.5074504E7|
|2021|Combined Heat and...|      6108302.0|
|2021|Combined Heat and...|      7389890.0|
|2020|Total Electric Po...|  1.555364328E9|
|2020|Electric Generato...|   8.26571344E8|
|2020|Electric Generato...|   6.60133031E8|
|2020|Combined Heat and...|    5.4882854E7|
|2020|Combined Heat and...|      6655509.0|
|2020|Combined Heat and...|      7121651.0|
|2019|Total Electric Po...|  1.444871622E9|
|2019|Electric Generato...|   7.49393222E8|
|2019|Electric Generato...|   6.24630909E8|
|2019|Combined Heat and...|    5.6789111E7|
|2019|Combined Heat and...|      6829030.0|
|2019|Combined Heat and...|      7229317.0|
|2018|Total Electric Po...|  1.401822577E9|
|2018|Electric Generato...|   7.

In [13]:
# Create a bank_data.py file and write the function to it. 
# This file will be stored in our pwd (/content/tests).
%%file total_energy.py

# Import findspark() and initialize. 
import findspark
findspark.init()

# Import other dependencies. 
from pyspark import SparkFiles
from pyspark.sql import SparkSession

# Create the import_data function. 
def import_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    energy = spark.read.csv('../drive/MyDrive/organised_Gen.csv', header = True, inferSchema = True)
    energy = energy.withColumnRenamed('TYPE OF PRODUCER', 'producer')
    energy = energy.withColumnRenamed('ENERGY SOURCE', 'source')
    energy = energy.withColumnRenamed('GENERATION (Megawatthours)', 'generated')
    energy.write.parquet('parquet_energy',mode='overwrite')
    p_energy = spark.read.parquet('parquet_energy')
    p_energy.createOrReplaceTempView('p_energy_data')
    return p_energy

def transform_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    transformed_df = spark.sql("""
    SELECT
        YEAR as year,
        MONTH as month,
        STATE as state, producer, source, generated
        FROM p_energy_data
        WHERE state == 'US-TOTAL' AND producer == 'Total Electric Power Industry'
        AND source == 'Total'
        """)
    return transformed_df

def query_data():
    spark = SparkSession \
    .builder \
    .appName("energy file read in") \
    .getOrCreate()
    q_df = spark.sql("""SELECT YEAR, (SUM(generated)/1000000)
            FROM p_energy_data 
            WHERE producer = 'Total Electric Power Industry' 
                AND source = 'Total' 
                AND state = 'US-TOTAL'
                AND YEAR != 2022
            Group By YEAR, State
            Order By YEAR DESC
                """)
    return q_df

Writing total_energy.py


In [14]:
# Create a test_bank_data.py file and write the test functions to it. 
# This file will be stored in our pwd (/content/tests).
%%file test_total_states_energy.py

from total_energy import (import_data, transform_data, query_data)

# Write the tests. 
def test_row_count_before_transform():
  df = import_data()
  assert df.count() == 496774

def test_column_count_before_transform():
  df = import_data()
  assert len(df.columns) == 7

def test_row_count_after_transform():
    df = transform_data()
    assert df.count() != 496774

def test_column_count_after_transform():
    df = transform_data()
    assert len(df.columns) != 7

def test_row_count_query():
    df = query_data()
    assert df.count() == 21

def test_column_count_query():
    df = query_data()
    assert len(df.columns) == 2

Writing test_total_states_energy.py


In [15]:
# Run the test_import_data.py file with pytest. 
!python -m pytest test_total_states_energy.py

Test session starts (platform: linux, Python 3.7.14, pytest 3.6.4, pytest-sugar 0.9.5)
rootdir: /content/tests, inifile:
plugins: typeguard-2.7.1, sugar-0.9.5

 test_total_states_energy.py ✓✓✓✓✓✓                              100% ██████████

Results (31.60s):
       6 passed
